In [251]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [252]:
df = pd.read_csv('./corona_data_sets/arbeitsmarktentwicklung.csv', converters={'ags2': str, 'ags5': str})
df = df.drop(['kreis', '_id'], 1)
df

,ags2,bundesland,ags5,kr_ka_au_202001,kr_ka_au_202002,kr_ka_au_202003,kr_ka_au_202004,kr_ka_au_202005,kr_ka_au_202006,kr_ka_au_202007,...,kr_rlb_202007,kr_rlb_202008,kr_rlb_202009,kr_rlb_202010,kr_rlb_202011,kr_rlb_202012,kr_rlb_202101,kr_rlb_202102,kr_rlb_202103,kr_rlb_202104
0,01,Schleswig-Holstein,01001,0,0,185,758,70,21,8,...,11528,11401,11211,11018,10941,10988,11066,11165.082749,11193.617211,11111.999397
1,01,Schleswig-Holstein,01002,5,4,344,1812,169,49,22,...,32153,31937,31493,31075,31018,30842,30738,30950.568347,30985.965972,30847.303052
2,01,Schleswig-Holstein,01003,0,0,370,1722,177,41,20,...,24463,24097,23613,23157,23179,23354,23491,23601.855952,23703.798710,23599.499787
3,01,Schleswig-Holstein,01004,0,0,142,642,55,15,5,...,8997,8893,8847,8730,8707,8667,8838,8870.375277,8954.945551,8825.197032
4,01,Schleswig-Holstein,01051,0,0,181,1046,80,34,15,...,10188,10002,9781,9613,9614,9706,9775,9817.517458,9890.821675,9906.267154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,16,Thüringen,16073,0,5,704,510,81,18,12,...,4985,4934,4824,4750,4751,4721,4727,4793.027079,4938.773207,4762.884313
397,16,Thüringen,16074,0,3,441,432,45,14,10,...,3324,3255,3181,3077,3022,2978,2994,3032.964802,3040.304609,3036.686847
398,16,Thüringen,16075,5,7,348,611,75,17,9,...,3445,3327,3270,3102,3060,3042,3060,3089.952200,3178.729884,3102.771109
399,16,Thüringen,16076,3,0,471,738,84,31,20,...,4126,4025,3949,3829,3826,3826,3891,3881.375161,3890.171929,3874.523927


In [253]:
df = pd.wide_to_long(df, i=['ags2', 'bundesland', 'ags5'], stubnames ='kr' , j = 'variable', sep='_',  suffix=r'\w+')
df

kr
ags2 bundesland         ags5  variable                 
01   Schleswig-Holstein 01001 ka_au_202001     0.000000
                              ka_au_202002     0.000000
                              ka_au_202003   185.000000
                              ka_au_202004   758.000000
                              ka_au_202005    70.000000
...                                                 ...
16   Thüringen          16077 rlb_202012    6196.000000
                              rlb_202101    6236.000000
                              rlb_202102    6248.782315
                              rlb_202103    6219.427429
                              rlb_202104    6228.279630

[89423 rows x 1 columns]

In [254]:
df = df.reset_index(drop=False)
df 

,ags2,bundesland,ags5,variable,kr
0,01,Schleswig-Holstein,01001,ka_au_202001,0.000000
1,01,Schleswig-Holstein,01001,ka_au_202002,0.000000
2,01,Schleswig-Holstein,01001,ka_au_202003,185.000000
3,01,Schleswig-Holstein,01001,ka_au_202004,758.000000
4,01,Schleswig-Holstein,01001,ka_au_202005,70.000000
...,...,...,...,...,...
89418,16,Thüringen,16077,rlb_202012,6196.000000
89419,16,Thüringen,16077,rlb_202101,6236.000000
89420,16,Thüringen,16077,rlb_202102,6248.782315
89421,16,Thüringen,16077,rlb_202103,6219.427429


In [255]:
set(df['variable'])

{'al_201901',
 'al_201902',
 'al_201903',
 'al_201904',
 'al_201905',
 'al_201906',
 'al_201907',
 'al_201908',
 'al_201909',
 'al_201910',
 'al_201911',
 'al_201912',
 'al_202001',
 'al_202002',
 'al_202003',
 'al_202004',
 'al_202005',
 'al_202006',
 'al_202007',
 'al_202008',
 'al_202009',
 'al_202010',
 'al_202011',
 'al_202012',
 'al_202101',
 'al_202102',
 'al_202103',
 'al_202104',
 'alga_201901',
 'alga_201902',
 'alga_201903',
 'alga_201904',
 'alga_201905',
 'alga_201906',
 'alga_201907',
 'alga_201908',
 'alga_201909',
 'alga_201910',
 'alga_201911',
 'alga_201912',
 'alga_202001',
 'alga_202002',
 'alga_202003',
 'alga_202004',
 'alga_202005',
 'alga_202006',
 'alga_202007',
 'alga_202008',
 'alga_202009',
 'alga_202010',
 'alga_202011',
 'alga_202012',
 'alq_201901',
 'alq_201902',
 'alq_201903',
 'alq_201904',
 'alq_201905',
 'alq_201906',
 'alq_201907',
 'alq_201908',
 'alq_201909',
 'alq_201910',
 'alq_201911',
 'alq_201912',
 'alq_202001',
 'alq_202002',
 'alq_202003',

In [256]:
df["date"] = df.variable.apply(lambda x: x[-6:])
df["variable"] = df.variable.apply(lambda x: ''.join(x.split())[:-7].upper())

In [257]:
df['date'] = pd.to_datetime(df['date'], format = '%Y%m')
df = df.rename(columns={"kr": "value"})

In [258]:
set(df['variable'])

{'AL',
 'ALGA',
 'ALQ',
 'GA',
 'KA_AP',
 'KA_AU',
 'KA_RP',
 'KA_RU',
 'RLB',
 'SVB_AO',
 'SVB_WO',
 'UB'}

In [259]:
di = {'AL': 'unemployed', 
      'ALGA': 'unemployment_benefit_recipients',
      'ALQ': 'unemployment_rate',
     'GA': 'registerd_jobs',
     'KA_AP': 'displayed_short_time_work_people',
     'KA_AU': 'displayed_short_time_work_companies',
     'KA_RP': 'realized_short_time_work_people',
     'KA_RU': 'realized_short_time_work_companies',
     'RLB': 'unemployment_benefit_entitled',
     'SVB_AO': 'employees_social_security_at_residence' ,
     'SVB_WO': 'employees_social_security_at_residence' 'employees_social_security_at_work',
     'UB': 'underemployment_without_short_time _work'}
df = df.replace({"variable": di})
df

,ags2,bundesland,ags5,variable,value,date
0,01,Schleswig-Holstein,01001,displayed_short_time_work_companies,0.000000,2020-01-01
1,01,Schleswig-Holstein,01001,displayed_short_time_work_companies,0.000000,2020-02-01
2,01,Schleswig-Holstein,01001,displayed_short_time_work_companies,185.000000,2020-03-01
3,01,Schleswig-Holstein,01001,displayed_short_time_work_companies,758.000000,2020-04-01
4,01,Schleswig-Holstein,01001,displayed_short_time_work_companies,70.000000,2020-05-01
...,...,...,...,...,...,...
89418,16,Thüringen,16077,unemployment_benefit_entitled,6196.000000,2020-12-01
89419,16,Thüringen,16077,unemployment_benefit_entitled,6236.000000,2021-01-01
89420,16,Thüringen,16077,unemployment_benefit_entitled,6248.782315,2021-02-01
89421,16,Thüringen,16077,unemployment_benefit_entitled,6219.427429,2021-03-01


In [260]:
#df.to_csv('./final_dfs/for_modeling/labor_market_by_date_long.csv', index = False)

In [261]:
df = df.pivot(index=["ags2","ags5","date"], columns="variable", values="value")
df = df.reset_index(drop=False)


In [262]:
df

variable,ags2,ags5,date,displayed_short_time_work_companies,displayed_short_time_work_people,employees_social_security_at_residence,employees_social_security_at_residenceemployees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate
0,01,01001,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,819.0,6166.0,4275.0,11452.000000,1155.0,8.2
1,01,01001,2019-02-01,NaN,NaN,NaN,NaN,NaN,NaN,864.0,6054.0,4065.0,11506.000000,1176.0,7.8
2,01,01001,2019-03-01,NaN,NaN,44651.0,32144.0,NaN,NaN,837.0,6021.0,4109.0,11435.000000,1126.0,7.8
3,01,01001,2019-04-01,NaN,NaN,NaN,NaN,NaN,NaN,823.0,5932.0,4005.0,11440.000000,1030.0,7.6
4,01,01001,2019-05-01,NaN,NaN,NaN,NaN,NaN,NaN,863.0,6036.0,4048.0,11401.000000,1012.0,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11223,16,16077,2020-12-01,54.0,882.0,NaN,NaN,NaN,NaN,702.0,4147.0,3015.0,6196.000000,979.0,6.7
11224,16,16077,2021-01-01,170.0,865.0,NaN,NaN,NaN,NaN,681.0,NaN,3287.0,6236.000000,NaN,7.3
11225,16,16077,2021-02-01,51.0,252.0,NaN,NaN,NaN,NaN,680.0,NaN,3270.0,6248.782315,NaN,7.3
11226,16,16077,2021-03-01,16.0,63.0,NaN,NaN,NaN,NaN,749.0,NaN,3222.0,6219.427429,NaN,7.2


In [263]:
#df.to_csv('./final_dfs/for_modeling/labor_market_by_date_wide.csv', index = False)